# <center>Workflow for on-line GC and HPLC analysis in flow chemistry</center>


---
## Section 0: Imports, Paths, and Logging
---

In this section all the necessary python packages are imported, the path to this notebook and the logger for this notebook is set up.

In [1]:
# Activate autoreload to keep on track with changing modules
%reload_ext autoreload
%autoreload 2

# Import standard libraries
import logging
from pathlib import Path

# Import tools for data processing and analysis and DaRUS upload
from FAIRFlowChemistry.tools.logger_setup import setup_logger
from FAIRFlowChemistry.tools import initialize_dataset, reading_raw_data_widget, analyzing_raw_data_widget, DaRUS_upload

# Get root path of the notebook
root = Path.cwd()

# Setup logger
setup_logger( "log_faraday_efficency.log" )
logger = logging.getLogger( "main" )

---
## Section 1: Dataset and raw data
---
In this section the dataset as well as the to analyze raw data is choosen

In [2]:
DaRUS_subjects = ['Agricultural Sciences', 'Arts and Humanities', 'Astronomy and Astrophysics', 'Business and Management', 'Chemistry', 
                  'Computer and Information Science', 'Earth and Environmental Sciences', 'Engineering', 'Law', 'Mathematical Sciences', 
                  'Medicine, Health and Life Sciences', 'Physics', 'Social Sciences', 'Other']

init_data = initialize_dataset()
init_data.write_dataset( subjects = DaRUS_subjects )

In [29]:
## Definition of basic meta data of the project ##

init_data.title.value        = 'FAIRFlow: A Jupyter notebook-based platform for on-line analysis in flow chemistry '
init_data.description.value  = 'The aim of this project is to introduce the FAIR (findable, accessible, interoperable, reusable) principles into miniplant flow chemistry'
init_data.project.value      = 'Project B07'

# List with authors and their affiliation #
init_data.authors.value      = 'Richard Schömig, Jan Range, Samir Darouich, Maximilian Schmidt, Seda Yilmaz, Elias Klemm, Jürgen Pleiss' 
init_data.affiliations.value = 'University of Stuttgart, University of Stuttgart, University of Stuttgart, University of Stuttgart, University of Stuttgart, University of Stuttgart, University of Stuttgart'
init_data.identifier.value   = 'xxx-xxx-xxx-xxx, xxx-xxx-xxx-xxx, xxx-xxx-xxx-xxx, xxx-xxx-xxx-xxx, xxx-xxx-xxx-xxx, xxx-xxx-xxx-xxx, xxx-xxx-xxx-xxx'
init_data.contact_text.value = 'Jürgen Pleiss, juergen.pleiss@itb.uni-stuttgart.de'

init_data.related_publication.value = "test, https://id.loc.gov/authorities/subjects/sh2014001146.html"

init_data.topic_classification.value = "homogeneous catalysis (LCSH), https://id.loc.gov/authorities/subjects/sh2014001146.html"
init_data.keywords.value             = "polymer chemistry (Loterre Chemistry Vocabulary), https://skosmos.loterre.fr/ERC/en/page/?uri=http%3A%2F%2Fdata.loterre.fr%2Fark%3A%2F67375%2FERC-KCSKD4X9-P"

init_data.dataset_text.value = "datasets/final"

In [2]:
## Search for dataset and raw data ##

rrdw = reading_raw_data_widget()
rrdw.choose_data( root = root, dataset_directory = "datasets")

In [14]:
## Change initalizing species data!

In [1]:
import yaml
from FAIRFlowChemistry.core import Calibration, Data, Quantity, Experiment


with open("data/additional_data/species_data.yaml") as f:
    species_data = yaml.safe_load(f)
species_data

{'Carbon dioxide': {'chemical_formula': 'CO2',
  'calibration': {'concentrations': [0, 50], 'peak_areas': [0, 38653]},
  'correction_factor': 0.74,
  'electron_transfer': 2},
 'Carbon monoxide': {'chemical_formula': 'CO',
  'calibration': {'concentrations': [0.5, 1, 5],
   'peak_areas': [797, 1328, 7223]},
  'correction_factor': 1.0,
  'electron_transfer': 2},
 'Hydrogen': {'chemical_formula': 'H2',
  'calibration': {'concentrations': [5, 10, 20], 'peak_areas': [71, 153, 330]},
  'correction_factor': 1.01,
  'electron_transfer': 2},
 'Ethane': {'chemical_formula': 'C2H6',
  'calibration': {'concentrations': [0, 5], 'peak_areas': [0, 12168]},
  'correction_factor': None,
  'electron_transfer': 16},
 'Ethene': {'chemical_formula': 'C2H4',
  'calibration': {'concentrations': [0.5, 2, 3],
   'peak_areas': [1122, 4864, 7297]},
  'correction_factor': None,
  'electron_transfer': 12},
 'Methane': {'chemical_formula': 'CH4',
  'calibration': {'concentrations': [5, 10], 'peak_areas': [5727, 119

In [2]:
test = Experiment()

In [3]:
for species,item in species_data.items():

    # Create Calibration object and fit it to the given data
    calibration = Calibration(
        peak_areas=Data(
            quantity="Peak area", values=item["calibration"]["peak_areas"]
        ),
        concentrations=Data(
            quantity=Quantity.CONCENTRATION.value,
            values=item["calibration"]["concentrations"],
        ),
    )
    calibration.calibrate()
    
    test.add_to_species_data( species = species, 
                              chemical_formula = item["chemical_formula"],
                              calibration = calibration,
                              correction_factor = item["correction_factor"],
                              electron_transfer = item["electron_transfer"]
                               )

Calibration
├── id = bcef0a33-0954-40e8-84ce-812c2aa34286
├── peak_areas
│   └── Data
│       ├── id = 6dbccfbd-2c00-478e-89d9-f17a37cdad92
│       ├── quantity = Peak area
│       └── values = [0.0, 38653.0, ...]
├── concentrations
│   └── Data
│       ├── id = 7824671b-6ad3-48e5-a08e-75fda85486bd
│       ├── quantity = Concentration
│       └── values = [0.0, 50.0, ...]
├── regression_coefficients = [-4.466559424312742e-15, 0.0012935606550591155, ...]
└── degree = 1

Calibration
├── id = 66c51884-1dae-451e-b103-e6825139a3d4
├── peak_areas
│   └── Data
│       ├── id = 23ce97b3-17cc-461d-9611-007fb871847e
│       ├── quantity = Peak area
│       └── values = [797.0, 1328.0, 7223.0, ...]
├── concentrations
│   └── Data
│       ├── id = aac491a0-c4f5-4089-bb3b-e6da4fa87d5b
│       ├── quantity = Concentration
│       └── values = [0.5, 1.0, 5.0, ...]
├── regression_coefficients = [0.012656829292048657, 0.0006912740171292097, ...]
└── degree = 1

Calibration
├── id = abcb5f13-bdc7-4184-8

In [7]:
# widget that reads in PID and LINKS each measurement types to devices?

# or first read PID and then provide the possibility to link each measurement to a device from dropdown!

In [32]:
#str(root) = "c:/Users/darouich/OneDrive/Dokumente/"

e_chem = str(root)+'/data/Rohdaten/7.1_nm_OMC/Gammry_Data/230705-1-CV-PEG20-068-GTA25/GSTATIC.DTA'
mfm    = [str(root)+'/data/Rohdaten/7.1_nm_OMC/MFM_Data/1-CV-PEG20-068/MFM_1-CV-PEG20-068.csv',
          str(root)+'/data/Rohdaten/7.1_nm_OMC/MFM_Data/2-CV-PEG20-068-GTA25/GSS_2-CV-PEG20-068-GTA25_200_20c_1h.csv',
          str(root)+'/data/Rohdaten/7.1_nm_OMC/MFM_Data/3-CV-PEG20-068-GTA25/GSS_3-CV-PEG20-068-GTA25_200_20c_1h.csv' ]
gc     = [str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-1-CV-PEG20-068-GTA25/JH-1H 2023-07-05 11-45-17/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-1-CV-PEG20-068-GTA25/JH-1H 2023-07-05 11-45-17/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-1-CV-PEG20-068-GTA25/JH-1H 2023-07-05 11-45-17/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-1-CV-PEG20-068-GTA25/JH-1H 2023-07-05 11-45-17/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-1-CV-PEG20-068-GTA25/JH-1H 2023-07-05 11-45-17/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-1-CV-PEG20-068-GTA25/JH-1H 2023-07-05 11-45-17/NV-F0104.D/REPORT01.CSV',

        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-2-CV-PEG20-068-GTA25/JH-1H 2023-07-05 14-19-03/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-2-CV-PEG20-068-GTA25/JH-1H 2023-07-05 14-19-03/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-2-CV-PEG20-068-GTA25/JH-1H 2023-07-05 14-19-03/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-2-CV-PEG20-068-GTA25/JH-1H 2023-07-05 14-19-03/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-2-CV-PEG20-068-GTA25/JH-1H 2023-07-05 14-19-03/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-2-CV-PEG20-068-GTA25/JH-1H 2023-07-05 14-19-03/NV-F0104.D/REPORT01.CSV',

        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-3-CV-PEG20-068-GTA25/JH-1H 2023-07-05 15-59-41/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-3-CV-PEG20-068-GTA25/JH-1H 2023-07-05 15-59-41/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-3-CV-PEG20-068-GTA25/JH-1H 2023-07-05 15-59-41/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-3-CV-PEG20-068-GTA25/JH-1H 2023-07-05 15-59-41/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-3-CV-PEG20-068-GTA25/JH-1H 2023-07-05 15-59-41/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC/GC_Data/230705-3-CV-PEG20-068-GTA25/JH-1H 2023-07-05 15-59-41/NV-F0104.D/REPORT01.CSV']


e_chem = str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/Gammry_Data/230706-1-CV-PEG20-068-GTA25-washing/GSTATIC.DTA'
mfm    = [str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/MFM_Data/1-CV-PEG20-068-GTA25-washing/GSS_1-CV-PEG20-068-GTA25-washing_200_20c_1h.csv',
          str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/MFM_Data/2-CV-PEG20-068-GTA25-washing/GSS_2-CV-PEG20-068-GTA25-washing_200_20c_1h.csv',
          str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/MFM_Data/3-CV-PEG20-068-GTA25-washing/GSS_3-CV-PEG20-068-GTA25-washing_200_20c_1h.csv' ]
gc     = [str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-1-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 09-26-39/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-1-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 09-26-39/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-1-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 09-26-39/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-1-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 09-26-39/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-1-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 09-26-39/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-1-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 09-26-39/NV-F0104.D/REPORT01.CSV',

        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-2-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 11-05-18/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-2-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 11-05-18/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-2-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 11-05-18/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-2-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 11-05-18/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-2-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 11-05-18/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-2-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 11-05-18/NV-F0104.D/REPORT01.CSV',

        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-3-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 12-32-22/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-3-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 12-32-22/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-3-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 12-32-22/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-3-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 12-32-22/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-3-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 12-32-22/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/7.1_nm_OMC_washing/GC_Data/230706-3-CV-PEG20-068-GTA25-washing/JH-1H 2023-07-06 12-32-22/NV-F0104.D/REPORT01.CSV']

e_chem = str(root)+'/data/Rohdaten/9.2_nm/Gammry_Data/230705-1-CV-PEG35-052-GTA25/GSTATIC.DTA'
mfm    = [str(root)+'/data/Rohdaten/9.2_nm/MFM_Data/1-CV-PEG35-052-GTA25/GSS_1-CV-PEG35-052-GTA25_200_20c_1h.csv',
          str(root)+'/data/Rohdaten/9.2_nm/MFM_Data/2-CV-PEG35-052-GTA25/GSS_2-CV-PEG35-052-GTA25_200_20c_1h.csv',]
gc     = [str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-1-CV-PEG35-052-GTA25/JH-1H 2023-07-05 18-32-59/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-1-CV-PEG35-052-GTA25/JH-1H 2023-07-05 18-32-59/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-1-CV-PEG35-052-GTA25/JH-1H 2023-07-05 18-32-59/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-1-CV-PEG35-052-GTA25/JH-1H 2023-07-05 18-32-59/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-1-CV-PEG35-052-GTA25/JH-1H 2023-07-05 18-32-59/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-1-CV-PEG35-052-GTA25/JH-1H 2023-07-05 18-32-59/NV-F0104.D/REPORT01.CSV',

        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-2-CV-PEG35-052-GTA25/JH-1H 2023-07-05 20-04-01/NV-F0102.D/report00.CSV',
        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-2-CV-PEG35-052-GTA25/JH-1H 2023-07-05 20-04-01/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-2-CV-PEG35-052-GTA25/JH-1H 2023-07-05 20-04-01/NV-F0103.D/report00.CSV',
        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-2-CV-PEG35-052-GTA25/JH-1H 2023-07-05 20-04-01/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-2-CV-PEG35-052-GTA25/JH-1H 2023-07-05 20-04-01/NV-F0104.D/report00.CSV',
        str(root)+'/data/Rohdaten/9.2_nm/GC_Data/230705-2-CV-PEG35-052-GTA25/JH-1H 2023-07-05 20-04-01/NV-F0104.D/REPORT01.CSV']

e_chem = str(root)+'/data/raw_data/01_EChem/CAD14-Cu@AB/GSTATIC.DTA'
mfm    = [str(root)+'/data/raw_data/03_MFM/CAD14-Cu@AB/Bench-2h-GSS_CAD14-Cu@AB_200_50c_24h_truncated.csv']
gc     = [str(root)+'/data/raw_data/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/report00.CSV',
        str(root)+'/data/raw_data/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0102.D/REPORT01.CSV',
        str(root)+'/data/raw_data/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/report00.CSV',
        str(root)+'/data/raw_data/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0103.D/REPORT01.CSV',
        str(root)+'/data/raw_data/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/report00.CSV',
        str(root)+'/data/raw_data/02_GC/CAD14-Cu@AB/JH-1H 2023-02-06 10-00-18/NV-F0104.D/REPORT01.CSV']
calib  = str(root)+'/data/additional_data/calibration.json'
correc = str(root)+'/data/additional_data/correction_factors.json'
farada = str(root)+'/data/additional_data/faraday_coefficients.json'

rrdw.galvano_files.value = [e_chem]
rrdw.MFM_files.value   = mfm
rrdw.GC_files.value    = gc
rrdw.calib_files.value = [calib]
rrdw.correction_files.value = [correc]
rrdw.faraday_files.value    = [farada]

---
## Section 2: Analysis of data
---
In this section the raw data of the above choosen dataset is analyzed (if you change the dataset above, then reexecute this cell)

In [8]:
# Provide a typical retention time dictionary to pre assign retention times 

typical_retention_time = {"Hydrogen": 1.7, "Carbon dioxide": 3.0, "Carbon monoxide": 13.6, 
                          "Methane": 3.6, "Ethene": 6.0, "Ethane": 7.1}

ardw = analyzing_raw_data_widget()
ardw.choose_experiment( dataset = rrdw.dataset, 
                        dataset_path = rrdw.dataset_dropdown.value, 
                        typical_retention_time = typical_retention_time)


GC measurement already contains an assignment. Use this assignment rather than the default dictionary.



---
## Section 3: Upload of data to DaRUS
---
In this section the above choosen dataset, containing the processed as well as the raw data, is uploaded to DaRUS

In [9]:
# Predefined list of all SFB1333 dataverses. If a new dataverse wants to be appended, just add in the list
sfb1333_dataverse_list = [ "sfb1333-"+group_name for group_name in ["hansen-gross", "grabowski", "sarkar", "tallarek", "peters", "schmitz", "inf", "pluhackova", "giesselmann", "klemm", "schlaich", "traa", "naumann", "lotsch", "vanSlageren", "holm", "fyta", "estes", "laschat", "bruckner", "bill", "ludwigs", "dyballa", "ringenberg", "kaestner", "Buchmeiser", "sottmann"] ]

Darus_upload = DaRUS_upload()

Darus_upload.DaRUS( dataset = ardw.dataset, 
                    dataset_path = rrdw.dataset_dropdown.value,
                    dataverse_list = sfb1333_dataverse_list )

Darus_upload.api_token_text.value = "4afecd82-c92d-4935-b786-2225af43531e"